# Hybrid Search with Elasticsearch and local embeddings

## Preconditions

To make this notebook work, you need to have an elasticsearch service (> `v8.9.0`) running on [http://localhost:9200](http://localhost:9200). To use the Reciprocal Rank Fusion in the byrid search (in the end of the notbook), you need to have a Platinum/Enterprise subscription [https://www.elastic.co/subscriptions](https://www.elastic.co/subscriptions)


## Helper Methods

In [28]:
def show_chunks(docs):
    for doc in docs:
        # check if doc is tuple
        if isinstance(doc, tuple):
            doc, score = doc
        else:
            score = None
        print("Page", doc.metadata["page"], ":")
        print(doc.page_content.replace('\n', ' '))
        if score:
            print("Score:", score)
        print(100 * "-")


## Load and chunk Documents

In [29]:
from pathlib import Path

raw_data_file = Path.cwd() / "sbb-geschaeftsbericht-2023.pdf"

In [30]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader(str(raw_data_file))
chunks = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0))
print("total chunks: ", len(chunks))
print('-' * 100)
print("Example:")
chunks[25]


total chunks:  970
----------------------------------------------------------------------------------------------------
Example:


Document(page_content='Geschäftsbericht 20238Konzernlagebericht\nDas Jahr 2023 in Zahlen. \n11,26\xa0Milliarden Franken \nSchulden Schuldendeckungsgrad: 7,82\n2700\xa0Sendungen Ein weiterer Anstieg der Verschuldung konnte gestoppt \nwerden (2022: 11,44\xa0Milliarden Franken). Doch der \nSpardruck bleibt hoch. Für eine nachhaltige Finanzie -\nrung wird die SBB  bis 2030 rund 6\xa0Milliarden Franken \nweniger ausgeben.Der Schuldendeckungsgrad misst, wie viele Jahre das laufende Ergebnis erzielt werden müsste, um die', metadata={'source': '/home/ursin/development/hybrid-search-demo/sbb-geschaeftsbericht-2023.pdf', 'page': 7})

## Create Embeddings using an Opensource Embedding Model

In [31]:
from langchain_community.embeddings import HuggingFaceEmbeddings

e5_multilingual_embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
e5_multilingual_embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='intfloat/multilingual-e5-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## Create Embedding from Question

In [32]:
query = "Wie viel weniger will die SBB bis 2030 ausgeben?"
e5_multilingual_embedding_model.embed_query(query)

[-0.020724598318338394,
 0.003353245323523879,
 -0.04718921333551407,
 -0.04045533016324043,
 0.026072410866618156,
 -0.018294258043169975,
 -0.01445701438933611,
 0.02227230742573738,
 0.06754158437252045,
 0.001783400890417397,
 0.0425213947892189,
 0.030309047549962997,
 -0.012533361092209816,
 -0.02684326469898224,
 -0.026066487655043602,
 -0.05336495116353035,
 0.023507127538323402,
 0.00879379641264677,
 -0.012215628288686275,
 -0.020137257874011993,
 0.03075406886637211,
 -0.04414905235171318,
 -0.01693112961947918,
 -0.000751925224903971,
 -0.007161107379943132,
 0.011271339841187,
 -0.032673366367816925,
 -0.019964460283517838,
 -0.007265779655426741,
 0.01016951072961092,
 -0.0591048002243042,
 -0.0019779279828071594,
 -0.043426934629678726,
 -0.020695149898529053,
 -0.020205454900860786,
 0.011324039660394192,
 0.05723607540130615,
 0.029726767912507057,
 -0.04964124783873558,
 0.01168445497751236,
 -0.027545282617211342,
 0.026116862893104553,
 -0.003331550629809499,
 -0.01

## Setup Connection to Elasticsearch

In [33]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

es_vector_search = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="test_index_vector_only",
    embedding=e5_multilingual_embedding_model
)

es_vector_search.add_documents(chunks)

['26eeabbd-4a0a-4518-8dbb-cfa5bc9543ad',
 '75457f40-29e5-4449-bb22-4d5ca6e0049e',
 'fbf759b3-b4d7-4be0-9635-8809d2c9e107',
 '9d56048f-20c9-43ca-960c-b97f3e03b0ec',
 '52fc3b3b-6ba0-4fd8-a098-b59d4e4ca318',
 '14116132-fed1-4598-ac5c-1e7561eb3904',
 '7cfde885-ce01-429e-9d98-3cad6f9dc071',
 'bbd0eb84-0601-4cb5-8147-835ea930c521',
 '556a1905-8f4a-4a8e-8a54-6f8719ce4aa3',
 'd3cebe87-257b-4fd7-b502-cd490810b127',
 'e923b133-2228-41f4-98c3-6cc186a7978b',
 '3b6b0533-6c4a-4127-a33b-171a12be9a44',
 'c1a788e8-a546-45ae-8740-0d7dba3b171d',
 '5af0b09f-fa02-4b71-96a9-0cd98a725c60',
 '7591a35e-bbdb-495d-9040-ec73e234aa92',
 'cdfc0f1a-0193-478e-861c-f82e3a7c61a9',
 'c4016240-c0d2-4c4d-9b5e-231c0ed32e08',
 'e94673ca-5a88-400e-af8f-6ded22df3368',
 '41d56db9-1f07-4bff-856a-72ca9b2ac01f',
 'aa1f3b4e-ac6b-4c78-b9f8-044c53dd069d',
 '73f7be14-b870-4ead-a3e0-c883e76371e7',
 'a725160e-a9e6-4a82-9712-3a8bca5dce07',
 '604b64cb-6245-4517-a42f-e8516abbc179',
 '651a0ed8-1370-4146-9e6a-3aba9a7c2e0f',
 '4b45bc44-b52c-

## Search with native Elasticsearch similarity search

In [34]:
similar_documents = es_vector_search.similarity_search_with_score(query, k=4)
show_chunks(similar_documents)

Page 45 :
ten. Trotz eines positiven Konzernergebnisses bleibt der Spar- und Effizienzdruck hoch. Das positive Jahresergeb - nis reicht noch nicht aus, um den notwendigen Schulden - abbau zu erreichen. Für eine nachhaltig finanzierte  SBB  sind jährlich rund 500 Millionen Franken Gewinn nötig. Nur  so können die Schulden abgebaut und anstehende In - vestitionen, unter anderem in neues Rollmaterial und in  Bahnhöfe, getätigt werden. Stabilisierungspaket – nachhaltige   Finanzierung bis 2030.
Score: 0.93489087
----------------------------------------------------------------------------------------------------
Page 45 :
Gesunde Finanzen sind für die Zukunft der SBB  essen - ziell. Das Verschuldungsniveau muss gesenkt werden  können, die Zinslast tragbar sein und das Eignerziel zum Schuldendeckungsgrad erreicht werden. Um dies zu er - wirken, leistet die  SBB  selbst einen grossen Beitrag, in - dem sie bis 2030 rund sechs Milliarden Franken weniger  ausgibt. Erträge aus Immobilien sichern 

## Search with a hybrid approach
To understand this approach better, have a look at: https://learn.microsoft.com/en-us/azure/search/hybrid-search-ranking


In [35]:
es_hybrid_search = ElasticsearchStore.from_documents(
    chunks, 
    e5_multilingual_embedding_model, 
    es_url="http://localhost:9200", 
    index_name="test_index_hybrid",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        hybrid=True,
    )
)

The following cell will only work if you have a paid ES version with supports Reciprocal Rank Fusion (RRF). Alterantively, you can build it yourself for example with the langchain **EnsembleRetriever**. Have a look at [https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

In [36]:
similar_documents = es_hybrid_search.similarity_search(query, k=3)
show_chunks(similar_documents)

AuthorizationException: AuthorizationException(403, 'security_exception', 'current license is non-compliant for [Reciprocal Rank Fusion (RRF)]')